# Fixing triangular meshes with trimesh

Consider: 
* The function `txttoarray` will only work for txt files inside the folder this notebook is in.
* Mesh will be made watertight by filling holes, deleting repeted vertices and faces, and by merging vertices that are close together. This distance can be toggled through the variable `merge_tolerance`.

In [1]:
%reset -f
import time
import numpy as np
import trimesh


def txttoarray(filename):
    file = open(filename, "r").read().split("\n")
    datos=[]
    for i in range(len(file)):
        datos.append(file[i].split())
    return(datos)

######################### #########################
######################### #########################

toc = time.time()
    
# Replace "surf_d02_stern.face" and "vert" with filename (txt extention)
face=txttoarray("Meshing_1fni/1fni_pH10.2_d08.face")
face=[i for i in face if i!=[]]

vert=txttoarray("Meshing_1fni/1fni_pH10.2_d08.vert")
vert=[i for i in vert if i!=[]]

vert_txt=np.float_(vert) ; vert_norm=vert_txt[:,3:6] ; vert=vert_txt[:,:3]
face=np.int_(face); face= face[:,:3]-1

######################### #########################
######################### #########################
    
mesh = trimesh.Trimesh(vertices=vert,faces=face,vertex_normals=vert_norm,validate=True,process=True)
mesh.fill_holes()
mesh.process()

while not mesh.is_watertight:
    merge_tolerance = 0.005
    needy_faces     = trimesh.repair.broken_faces(mesh)
    for vert_nf in mesh.faces[needy_faces]:
        for nf in vert_nf:
            for c,check in enumerate(np.linalg.norm(mesh.vertices[vert_nf]-mesh.vertices[nf],axis=1)):
                if (check<merge_tolerance) & (0<check):
                    mesh.vertices[nf]=mesh.vertices[vert_nf[c]]
                    
mesh.fill_holes()
mesh.process()
            
                
tic = time.time()
print("Total time:",round(tic-toc,3),"[s]")

Total time: 0.652 [s]


In [2]:
print(mesh)

<trimesh.Trimesh(vertices.shape=(33884, 3), faces.shape=(67740, 3))>


In [3]:
mesh_split = mesh.split()
print(mesh)
print(mesh_split)
print("len de mesh_split: ",len(mesh_split))

<trimesh.Trimesh(vertices.shape=(33884, 3), faces.shape=(67740, 3))>
[<trimesh.Trimesh(vertices.shape=(31822, 3), faces.shape=(63640, 3))>
 <trimesh.Trimesh(vertices.shape=(510, 3), faces.shape=(1016, 3))>
 <trimesh.Trimesh(vertices.shape=(318, 3), faces.shape=(632, 3))>
 <trimesh.Trimesh(vertices.shape=(108, 3), faces.shape=(212, 3))>
 <trimesh.Trimesh(vertices.shape=(752, 3), faces.shape=(1500, 3))>
 <trimesh.Trimesh(vertices.shape=(178, 3), faces.shape=(352, 3))>
 <trimesh.Trimesh(vertices.shape=(196, 3), faces.shape=(388, 3))>]
len de mesh_split:  7


In [4]:
mesh_ses = mesh_split[0]
print(mesh_ses)

<trimesh.Trimesh(vertices.shape=(31822, 3), faces.shape=(63640, 3))>


In [5]:
mesh_ses.show()

In [6]:
mesh_ses_faces = mesh_ses.faces + 1
mesh_ses_vert = mesh_ses.vertices
print(mesh_ses_faces, mesh_ses_vert)

[[  125   505   738]
 [  414 20929 27649]
 [20929 16974 27649]
 ..., 
 [17094 20076 21637]
 [ 5547 18317 21634]
 [10276 13572 21775]] [[ 20.48   37.418  18.3  ]
 [ 20.48   27.721  28.802]
 [ 30.72   22.14   12.713]
 ..., 
 [ 49.635  27.503   4.073]
 [ 49.635  38.996 -11.799]
 [ 20.963  25.225   7.31 ]]


In [7]:
np.savetxt("Meshing_1fni/1fni_split/1fni_pH10.2_d08_split.face",mesh_ses_faces,fmt="%i")
np.savetxt("Meshing_1fni/1fni_split/1fni_pH10.2_d08_split.vert",mesh_ses_vert)

In [8]:
#mesh: face - vert 1a0j
#d01: 7384 - 3690
#d02: 15684 - 7842
#d04: 32524 - 16260
#d08: 66580 - 33286
#d16: 135644 - 67816

#mesh: face - vert 1smd
#d01: 13896 - 6944
#d02: 31080 - 14650
#d04: 61240 - 30616
#d08: 125016 - 62500
#d16: 253844 - 135878

#mesh: face - vert 1fni pH8.0
#d01: 7024 - 3514
#d02: 15052 - 7528
#d04: 30904 - 15454
#d08: 63756 - 31880
#d16: 129636 - 64818

#mesh: face - vert 1fni pH10.2
#d01: 7008 - 3506
#d02: 15028 - 7516
#d04: 30820 - 15412
#d08: 63640 - 31822
#d16: 129404 - 64702